In [3]:
#pip install mediapipe

In [1]:
import cv2
import numpy as np
import pandas as pd
import math
import os
import mediapipe as mp


mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)


def calculate_angle(p1, p2, p3):
    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p2)
    angle = np.arccos(
        np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
    )
    return np.degrees(angle)


video_folder = "final_videos"
video_files = sorted([f for f in os.listdir(video_folder) if f.endswith(".mp4")])

dataset = []

for video_index, video_file in enumerate(video_files):
    video_path = os.path.join(video_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    prev_keypoints = None
    prev_velocity = None
    prev_time = None

    frame_index = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            keypoints = []
            for landmark in results.pose_landmarks.landmark:
                keypoints.append([landmark.x, landmark.y, landmark.z])

            height, width, _ = frame.shape
            keypoints = np.array(keypoints) * np.array([width, height, 1])  

            current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0 

            
            velocity = None
            acceleration = None

            if prev_keypoints is not None and prev_time is not None:
                time_diff = current_time - prev_time

                if time_diff > 0:
                    velocity = (keypoints - prev_keypoints) / time_diff
                    if prev_velocity is not None:
                        acceleration = (velocity - prev_velocity) / time_diff

            
            angles = {
                25: ("left_knee_angle", calculate_angle(keypoints[23], keypoints[25], keypoints[27])),  
                26: ("right_knee_angle", calculate_angle(keypoints[24], keypoints[26], keypoints[28])),  
                11: ("left_elbow_angle", calculate_angle(keypoints[9], keypoints[11], keypoints[13])),  
                12: ("right_elbow_angle", calculate_angle(keypoints[10], keypoints[12], keypoints[14]))  
            }

            
            for i, point in enumerate(keypoints):
                data_entry = {
                    "video_id": video_index + 1,  
                    "frame": frame_index,
                    "keypoint_id": i,
                    "x": point[0],
                    "y": point[1],
                    "confidence": 1.0,  
                    "velocity_x": velocity[i][0] if velocity is not None else None,
                    "velocity_y": velocity[i][1] if velocity is not None else None,
                    "acceleration_x": acceleration[i][0] if acceleration is not None else None,
                    "acceleration_y": acceleration[i][1] if acceleration is not None else None
                }

               
                if i in angles:
                    angle_name, angle_value = angles[i]
                    data_entry[angle_name] = angle_value

                dataset.append(data_entry)

            prev_keypoints = keypoints.copy()
            prev_velocity = velocity.copy() if velocity is not None else None
            prev_time = current_time
        frame_index += 1

    cap.release()


df = pd.DataFrame(dataset)
df.to_csv("mediapipe_motion_dataset.csv", index=False)

print("The data was saved")


The data was saves
